In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import numpy as np

In [ ]:
police_df = pd.read_csv('../data/police.csv')
police_df = police_df.drop(columns = 'Unnamed: 0')
police_df

In [ ]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
zipcodes.head( )

In [ ]:
print(police_df.loc[3,'Latitude'])
print(type(police_df.loc[3,'Latitude']))

In [ ]:
isnan(police_df.loc[0,'Latitude'])

In [ ]:
geo = []

for row, value in police_df.iterrows():
    if not isnan(value.Latitude):
        coord = Point(value.longitude,value.Latitude)
        geo.append(coord)
    else:
        geo.append(None)

In [ ]:
len(geo)

In [ ]:
police_df['geometry'] = geo

In [ ]:
police_gdf = gpd.GeoDataFrame(police_df)
police_gdf

In [ ]:
police_gdf.crs = zipcodes.crs

### Saving the GeoDataFrame so that I don't have to run 20 minutes of code again

In [ ]:
police_gdf.to_csv('../data/police_geo.csv')

In [ ]:
police_loc = police_gdf.loc[police_gdf.geometry == None]
#police_loc = police_loc.reset_index(drop=True)
police_loc

In [ ]:
police_gdf = police_gdf.replace({np.nan:None})

In [ ]:
type(police_gdf.loc[0,'geometry'])

In [ ]:
police_gdf = pd.read_csv('../data/police_geo.csv')
police_gdf

In [ ]:
police_gdf = gpd.GeoDataFrame(police_gdf)
police_gdf.set_geometry('Mapped_Location')
police_gdf.crs = zipcodes.crs

In [ ]:
area_center = [36.23245257407244, -86.77676762469858]

In [ ]:
police_map = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(zipcodes).add_to(police_map)

marker_cluster = MarkerCluster().add_to(police_map)

#remember for Folium locations, lat is listed first!!
i=0
for row_index, row_values in police_loc.iterrows():
    print(i)
    loc = [row_values['Latitude'], row_values['longitude']]
        
    if row_index < 5988325:
        icon=folium.Icon(color="red",icon="phone", prefix='fa')
    else:
        icon=folium.Icon(color='blue',icon='handcuffs', prefix='fa')
            
    marker = folium.Marker(
        location = loc, 
        icon = icon) 
        
    marker.add_to(police_map)
    i += 1
#end

In [ ]:
police_map